# SECOND TASK - GET LATITUDES AND LONGITUDES

**In this first line, I import all the libraries needed for the assignment.**

In [3]:
# library to handle data in a vectorized manner
import numpy as np 

# library for data analsysis
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# import needed libraries for web scraping
from bs4 import BeautifulSoup
import lxml
import html5lib
import requests

# map rendering library
import folium 

# convert an address into latitude and longitude values
from geopy.geocoders import Nominatim 

print('Libraries imported.')

Libraries imported.


**In the following lines I write the entire code to get the first table.**

***Note***: as I've already explained in the first task how I did the table, there is not need to do it again. However, the first task can be review in the following link:*

[Task 1](https://github.com/JulioE-Lopez/Coursera_Capstone/blob/master/Toronto's%20Assignment-Task%201.ipynb)

In [4]:
!wget -q -O 'toronto_postcode.html' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
print('Data downloaded!')

Data downloaded!


In [5]:
with open('toronto_postcode.html') as html_file:
    soup = BeautifulSoup(html_file,'lxml')

In [6]:
new_table = soup.find('table',{'class':'wikitable sortable'})

In [7]:
values_post = new_table.findAll('td')[0::3]

PostalCode = []
for value in values_post:
    PostalCode.append(value.text)    

In [8]:
values_bor = new_table.findAll('td')[1::3]

Borough = []
for value in values_bor:
    Borough.append(value.text) 

In [9]:
values_nei = new_table.findAll('td')[2::3]

Neighborhood = []
for value in values_nei:
    value = value.text
    value = value.split("\n")
    value = value[0]
    Neighborhood.append(value)

In [10]:
df_toronto = pd.DataFrame()
df_toronto['PostalCode'] = PostalCode
df_toronto['Borough'] = Borough
df_toronto['Neighborhood'] = Neighborhood

In [11]:
df_toronto.drop(df_toronto.index[borough_mask], inplace=True)
df_toronto.reset_index(drop=True, inplace=True)

In [12]:
neighborhood_mask = df_toronto.index[df_toronto['Neighborhood'] == 'Not assigned']

for idx in neighborhood_mask:
    df_toronto['Neighborhood'][idx] = df_toronto['Borough'][idx]

In [14]:
lambda_nei = lambda x : '%s' % ', '.join(x)
lambda_bo = lambda x : set(x).pop()

new = df_toronto.groupby('PostalCode')
new_neighborhood = new['Neighborhood'].apply(lambda_nei)
new_borough = new['Borough'].apply(lambda_bo)

columns = list(zip(new_borough.index, new_borough, new_neighborhood))
df_toronto_new = pd.DataFrame(columns)
df_toronto_new.columns = ['PostalCode','Borough','Neighborhood']

In [15]:
print(df_toronto_new.shape)
df_toronto_new.head()

(103, 3)


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


**In the next line I open the csv file with the latitudes and longitudes of the Toronto postalcodes into a pandas dataframe.** 

In [16]:
toronto_latlon = pd.read_csv('https://cocl.us/Geospatial_data')
toronto_latlon.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


**I double check that the length of this new dataframe is the same as the one I shaped.**

In [17]:
toronto_latlon.shape

(103, 3)

**In this line I merge the two dataframes, and drop the extra column: Postal Code**

In [18]:
df_toronto_latlon = pd.concat([df_toronto_new, toronto_latlon],axis=1)
df_toronto_latlon.drop(['Postal Code'], axis = 1, inplace = True)

## FINAL RESULT

In [19]:
print(df_toronto_latlon.shape)
df_toronto_latlon.head()

(103, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


----------------
*The dataframe was parsed for the following wikipedia page:*

[Toronto Postal Codes](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M)